<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/OnePage_Insight_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# STEP 0: Install dependency
!pip install xlsxwriter

# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# STEP 2: File paths
ros_path = '/content/drive/MyDrive/SCI_Works/One_Page/OnePage_ROS_Cleaned.xlsx'
mapping_path = '/content/drive/MyDrive/SCI_Works/One_Page/MappingRules_Cleaned.xlsx'

# STEP 3: Import
import pandas as pd

# STEP 4: Load sheets
may_df = pd.read_excel(ros_path, sheet_name="May25", header=0)
ytd_df = pd.read_excel(ros_path, sheet_name="AprMay25_YTD", header=0)
map_df = pd.read_excel(mapping_path, header=0)

# STEP 5: Merge only mapped
may_joined = pd.merge(may_df, map_df, on="Customer", how="left").dropna(subset=["Group"])
ytd_joined = pd.merge(ytd_df, map_df, on="Customer", how="left").dropna(subset=["Group"])

# STEP 6: Calculate TTL %
for df in [may_joined, ytd_joined]:
    df['TTL %'] = df.apply(lambda row: f"{round((row['Actual']/row['Plan'])*100)}%" if row['Plan'] != 0 else "-", axis=1)

# STEP 7: Add total row
def add_total(df, has_ctype=True):
    total_actual = df['Actual'].sum()
    total_plan = df['Plan'].sum()
    ttl_percent = f"{round((total_actual / total_plan)*100)}%" if total_plan != 0 else "-"
    row = {'Group': 'Total', 'Customer': '', 'Actual': total_actual, 'Plan': total_plan, 'TTL %': ttl_percent}
    if has_ctype:
        row['C_type'] = ''
    return pd.concat([df, pd.DataFrame([row])], ignore_index=True)

# STEP 8: Final table
may_result = may_joined[['Group', 'C_type', 'Customer', 'Actual', 'Plan', 'TTL %']]
may_result = add_total(may_result, has_ctype=True)

ytd_result = ytd_joined[['Group', 'Customer', 'Actual', 'Plan', 'TTL %']]
ytd_result = add_total(ytd_result, has_ctype=False)

# STEP 9: Export
output_path = '/content/drive/MyDrive/SCI_Works/One_Page/OnePage_CustomerSummary.xlsx'
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    may_result.to_excel(writer, sheet_name='May25', index=False)
    ytd_result.to_excel(writer, sheet_name='AprMay25_YTD', index=False)

# STEP 10: Preview
print("✅ Done. May Result:")
may_result.tail(5)
from IPython.display import display
print("✅ May Summary:")
display(may_result.tail(5))

print("📊 YTD Summary:")
display(ytd_result.tail(5))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Done. May Result:
✅ May Summary:


,Group,C_type,Customer,Actual,Plan,TTL %
16,SSK,Scroll,SSK,48,0,-
17,Taiwan,Scroll,TECO (SANYU),128,344,37%
18,SSK,Scroll,TICA (SSK),1520,2112,72%
19,SSK,Scroll,XINGXING,0,0,-
20,Total,,,3858,4440,87%


📊 YTD Summary:


,Group,Customer,Actual,Plan,TTL %
17,SSK,SSK,48,0,-
18,Taiwan,TECO (SANYU),128,624,21%
19,SSK,TICA (SSK),2240,4144,54%
20,SSK,XINGXING,0,112,0%
21,Total,,5258,8247,64%
